In [1]:
import pandas as pd

In [2]:
url = "https://github.com/bigdata-icict/ETL-Dataiku-DSS/blob/master/SIM/cid10_tabela_capitulos.csv?raw=true"
df_capitulos = pd.read_csv(url, sep=';')

url = "https://github.com/bigdata-icict/ETL-Dataiku-DSS/blob/master/SIM/cid10_tabela_grupos.csv?raw=true"
df_grupos = pd.read_csv(url, sep=';')

url = "https://github.com/bigdata-icict/ETL-Dataiku-DSS/blob/master/SIM/CID-10-CATEGORIAS.CSV.utf8?raw=true"
df_categorias = pd.read_csv(url, sep=';')

url = "https://github.com/bigdata-icict/ETL-Dataiku-DSS/blob/master/SIM/CID-10-SUBCATEGORIAS.CSV.utf8?raw=true"
df_subcategorias = pd.read_csv(url, sep=';')

In [3]:
df_grupos[df_grupos['codigo'] == 'X58']

,codigo,descricao,descricao_breve
2197,X58,Acidentes,Acidentes
2555,X58,Outras causas externas de traumatismos acidentais,Outras causas externas de traumatismos acidentais
2715,X58,Exposição acidental a outros fatores e aos não...,Exposição acidental a outr fatores e aos não e...
